In [2]:
import numpy as np
from scipy.stats import unitary_group

import matplotlib.pyplot as plt
np.set_printoptions(precision=3, suppress=True, linewidth=150) #print less digits of floats for readability
%matplotlib inline
%config InlineBackend.figure_format='retina' # makes the plots look nicer

Reck decomposition test

In [3]:
from strawberryfields.decomposition_tests import reck_decompose,reck_recompose
m = 6
U = unitary_group.rvs(m)
phases = reck_decompose(U)
U_out = reck_recompose(phases)
np.allclose(U, U_out, atol=1e-14, rtol=1e-14)

True

In [11]:
import numpy as np
from strawberryfields.program import Program
from strawberryfields.ops import Rgate,BSgate,sMZgate
from strawberryfields.engine import Engine


circuit = Program(m)

with circuit.context as q:
    for j in range(m-1):
        phi_j = phases['phi_ins'][j]
        Rgate(phi_j) | q[j+1]
        for k in range(j+1):
            n = j - k
            delta = phases['deltas'][n,k]
            sigma = phases['sigmas'][n,k]
            phi1 = sigma+delta
            phi2 = sigma-delta
            sMZgate(phi1,phi2) | (q[n],q[n+1])
    for j in range(m):
        zeta = phases['zetas'][j]
        Rgate(zeta) | q[j]

In [12]:
compiled_circuit = circuit.compile(compiler="gaussian_unitary")
commands  = compiled_circuit.circuit
S = commands[0].op.p[0] #this is how to get the symplectic transformation from the compiled circuit
from thewalrus.symplectic import sympmat
omega = sympmat(m)
# np.allclose(S @ omega @ S.T, omega) #omega is symplectic
U_new = S[:m,:m] + 1j*S[m:,:m] # this is how to get the unitary transformation from the symplectic transmformation
# U_new @ U_new.T.conj() #U is unitary

In [15]:
np.allclose(U, U_new, atol=1e-14, rtol=1e-14)

True

Clement decomposition test

In [16]:
from strawberryfields.decomposition_tests import clement_decompose,clement_recompose
m = 7
U = unitary_group.rvs(m)
phases = clement_decompose(U)
U_out = clement_recompose(phases)
np.allclose(U, U_out, atol=1e-14, rtol=1e-14)

True

In [18]:
circuit = Program(m)

with circuit.context as q:
    #upper left of interferometer
    for j in range(0,m-1,2):
        phi_j = phases['phi_ins'][j]
        Rgate(phi_j) | q[j]
        for k in range(j+1):
            n = j - k
            delta = phases['deltas'][n,k]
            sigma = phases['sigmas'][n,k]
            phi1 = sigma+delta
            phi2 = sigma-delta
            sMZgate(phi1,phi2) | (q[n],q[n+1])
            
    #diagonal phases
    for j in range(m):
        zeta = phases['zetas'][j]
        Rgate(zeta) | q[j]
    
    #lower right of interferometer
    for j in reversed(range(1,m-1,2)):
        for k in reversed(range(j+1)):
            n = m + k - j - 2
            delta = phases['deltas'][n,m-k-1]
            sigma = phases['sigmas'][n,m-k-1]
            phi1 = sigma+delta
            phi2 = sigma-delta
            sMZgate(phi1,phi2) | (q[n],q[n+1])
            
    for j in range(1,m-1,2):
        x = m - j - 1
        phi_j = phases['phi_outs'][x]
        Rgate(phi_j) | q[x]

In [19]:
compiled_circuit = circuit.compile(compiler="gaussian_unitary")
commands  = compiled_circuit.circuit
S = commands[0].op.p[0] #this is how to get the symplectic transformation from the compiled circuit
from thewalrus.symplectic import sympmat
omega = sympmat(m)
# np.allclose(S @ omega @ S.T, omega) #omega is symplectic
U_new = S[:m,:m] + 1j*S[m:,:m] # this is how to get the unitary transformation from the symplectic transmformation
# U_new @ U_new.T.conj() #U is unitary

In [20]:
np.allclose(U, U_new, atol=1e-14, rtol=1e-14)

True

Rectangle decomposition test

In [23]:
from strawberryfields.decomposition_tests import rectangle_compact_decompose,rectangle_compact_recompose
m = 10
U = unitary_group.rvs(m)
phases = rectangle_compact_decompose(U)
U_out = rectangle_compact_recompose(phases)
np.allclose(U, U_out, atol=1e-14, rtol=1e-14)

True

In [24]:
circuit = Program(m)

with circuit.context as q:
    for j in range(0,m-1,2):
        phi = phases['phi_ins'][j]
        Rgate(phi) | q[j]
    for layer in range(m):
        if (layer + m + 1) % 2 == 0:
            phi_bottom = phases['phi_edges'][m-1, layer]
            Rgate(phi_bottom) | q[m-1]
        for mode in range(layer % 2, m-1, 2):
            delta = phases['deltas'][mode, layer]
            sigma = phases['sigmas'][mode, layer]
            phi1 = sigma+delta
            phi2 = sigma-delta
            sMZgate(phi1,phi2) | (q[mode],q[mode+1])
    for j, phi_j in phases['phi_outs'].items():
        Rgate(phi_j) | q[j]

In [25]:
compiled_circuit = circuit.compile(compiler="gaussian_unitary")
commands  = compiled_circuit.circuit
S = commands[0].op.p[0] #this is how to get the symplectic transformation from the compiled circuit
from thewalrus.symplectic import sympmat
omega = sympmat(m)
# np.allclose(S @ omega @ S.T, omega) #omega is symplectic
U_new = S[:m,:m] + 1j*S[m:,:m] # this is how to get the unitary transformation from the symplectic transmformation
# U_new @ U_new.T.conj() #U is unitary

In [26]:
np.allclose(U, U_new, atol=1e-14, rtol=1e-14)

True